# YouTube
Tem como objetivo baixar videos no YouTube.

## Setup

In [1]:
from pytube import YouTube as PyTube, StreamQuery
from pathlib import Path
import pandas as pd
from IPython.display import clear_output
from functools import cache
import shlex

YOUTUBE_PATH = Path("./files/youtube")
YOUTUBE_PATH.mkdir(exist_ok=True)


class YouTube:
    DOWNLOAD_BLOCKS = 20

    def __init__(self, url: str):
        self._yt = PyTube(url=url, on_progress_callback=self._progress)
        self.url = url
        self.autor = self._yt.author
        self.titulo = self._yt.title
        self._filesize: int = 0

    def __str__(self) -> str:
        return f"<Youtube autor={shlex.quote(self.autor)} titulo={shlex.quote(self.titulo)} url={shlex.quote(self.url)}>"

    def __repr__(self) -> str:
        return str(self)

    @property
    def _streams(self) -> StreamQuery:
        @cache
        def wrapper():
            return self._yt.streams
        return wrapper()

    def _progress(self, chunk, file_handle: bytes, bytes_remaining: int):
        percent = ((self._filesize - bytes_remaining) * 100) / self._filesize
        block = int((percent * self.DOWNLOAD_BLOCKS) / 100)
        clear_output(wait=True)
        print("Progresso: [{0}] {1:.1f}%".format("#" * block + "-" * (20 - block), percent))

    def resolucoes(self, only_progressive: bool = False) -> pd.DataFrame:
        resolutions = self._streams
        if only_progressive:
            resolutions = resolutions.filter(progressive=True)
        df = pd.DataFrame([res.__dict__ for res in resolutions])
        return df
    
    def baixar_video(self, itag: int) -> str:
        video = self._streams.get_by_itag(itag)
        self._filesize = video.filesize
        return video.download(output_path=str(YOUTUBE_PATH))


## Execução
Executar os 3 passos abaixo na sequencia:

### 1. Digitar URL do video

In [2]:
youtube_url = "https://www.youtube.com/watch?v=BhF6mkQtObw&t=66s"
yt = YouTube(url=youtube_url)
yt

<Youtube autor='Copa Studio' titulo='Irmão do Jorel - Vovó Me Leva pra Aula - Completo (Oficial)' url='https://www.youtube.com/watch?v=BhF6mkQtObw&t=66s'>

### 2. Listar Resoluções

In [3]:
# Somente campos
", ".join(list(yt.resolucoes()))

'_monostate, url, itag, mime_type, codecs, type, subtype, video_codec, audio_codec, is_otf, bitrate, _filesize, _filesize_kb, _filesize_mb, _filesize_gb, is_dash, abr, fps, resolution, is_3d, is_hdr, is_live'

In [4]:
yt.resolucoes(only_progressive=True)[["itag", "mime_type", "video_codec", "audio_codec", "fps", "resolution", "abr"]]
# yt.resolucoes()[["itag", "mime_type", "video_codec", "audio_codec", "fps", "resolution", "abr"]]

,itag,mime_type,video_codec,audio_codec,fps,resolution,abr
0,17,video/3gpp,mp4v.20.3,mp4a.40.2,12,144p,24kbps
1,18,video/mp4,avc1.42001E,mp4a.40.2,24,360p,96kbps
2,22,video/mp4,avc1.64001F,mp4a.40.2,24,720p,192kbps


### 3. Baixar Video

In [5]:
yt.baixar_video(itag=22)

Progresso: [####################] 100.0%


'/home/douglaspands/workspace/vscode-notebook/files/youtube/Irmão do Jorel - Vovó Me Leva pra Aula - Completo (Oficial).mp4'